<a href="https://colab.research.google.com/github/iceshadow2107/AIDEA-Text-labeling-and-identification-of-agricultural-articles/blob/main/AICUP_2021_ver4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import monpa
from monpa import utils
import os
import csv

# 修改成第一階段以及private資料

只改檔案名即可(first_data是有label的檔案，final_data是Private測試集階段)(因為寫test跟train我會搞混......)

In [ ]:
first_data = {}
for file in os.listdir("./dataTrainComplete"):
    f = open("./dataTrainComplete/" + file, mode='r', encoding="utf-8")
    text = f.read()
    idx = os.path.splitext(file)[0]
    if idx not in first_data.keys():
        first_data[idx] = text

In [ ]:
final_data = {}
for file in os.listdir("./dataPrivateComplete"):
    f = open("./dataPrivateComplete/" + file, mode='r', encoding="utf-8")
    text = f.read()
    idx = os.path.splitext(file)[0]
    if idx not in final_data.keys():
        final_data[idx] = text

In [ ]:
chem = [line.strip() for line in open ('chem.txt', encoding='utf-8').readlines()]
crop = [line.strip() for line in open ('crop.txt', encoding='utf-8').readlines()]
pest = [line.strip() for line in open ('pest.txt', encoding='utf-8').readlines()]
def GROUP(article):
    group = []
    i = 0
    j = 0
    l = 0
    for i in range(len(chem)):
        word = str(chem[i])
        if word in article:
            group.append(word)
    for j in range(len(crop)):
        word = str(crop[j])
        if word in article:
            group.append(word)
    for k in range(len(pest)):
        word = str(pest[k])
        if word in article:
            group.append(word)
    return group

In [ ]:
for key in list(first_data.keys()):
    first_data[key] = str(GROUP(first_data[key]))

559


In [ ]:
for key in list(final_data.keys()):
    final_data[key] = str(GROUP(final_data[key]))

420


# 修改成第一階段以及private資料你想要輸出的檔名

只改輸出的檔案名即可(first_data是有label的檔案，final_data是Private測試集階段)(因為寫test跟train我會搞混......)

In [ ]:
with open('first_data_monpa.csv', mode='w', encoding="utf-8") as f:  
    writer = csv.writer(f)
    for k, v in first_data.items():
        writer.writerow([k, v])

In [ ]:
with open('final_data_monpa.csv', mode='w', encoding="utf-8") as f:  
    writer = csv.writer(f)
    for k, v in final_data.items():
        writer.writerow([k, v])

# 製作第一階段的Train label sample

In [ ]:
first_data = pd.read_csv('first_data_monpa.csv', header=None, index_col=0, squeeze=True).to_dict()

In [ ]:
TrainLabel = pd.read_csv('TrainLabel.csv', header=0, encoding='utf-8')
TrainLabel_test = TrainLabel['Test'].values.tolist()
TrainLabel_Reference = TrainLabel['Reference'].values.tolist()

In [ ]:
TrainLabel_test_text = []
TrainLabel_Reference_text = []
for item in TrainLabel_test:
    TrainLabel_test_text.append(str(first_data[item]))
for item in TrainLabel_Reference:
    TrainLabel_Reference_text.append(str(first_data[item]))

In [ ]:
pn = [1]*len(TrainLabel)

In [ ]:
TrainLabel_text_1 = pd.DataFrame({'Test':TrainLabel_test, 'Reference':TrainLabel_Reference, 'Test_text':TrainLabel_test_text, 'Reference_text':TrainLabel_Reference_text, 'P/N':pn})

In [ ]:
list_key_Label = list(first_data.keys())
LabellistA = [val for val in list_key_Label for i in range(len(list_key_Label))]
LabellistB = list_key_Label*len(list_key_Label)

In [ ]:
TrainLabel_test_text = []
TrainLabel_Reference_text = []
for item in LabellistA:
    TrainLabel_test_text.append(str(first_data[item]))
for item in LabellistB:
    TrainLabel_Reference_text.append(str(first_data[item]))

In [ ]:
d = [str(list(first_data.keys())[i])*2 for i in range(len(first_data.keys()))]
LabellistC = list(str(LabellistA[i])+str(LabellistB[i]) for i in range(len(LabellistA)))
TrainLabel_combo = list((str(TrainLabel_test[i])+str(TrainLabel_Reference[i]) for i in range(len(TrainLabel)))) + d

In [ ]:
PN = []
for i in range(len(LabellistC)):
    if LabellistC[i] in TrainLabel_combo:
        PN.append(1)
    else:
        PN.append(0)

In [ ]:
TrainLabel_text_all = pd.DataFrame({'Test':LabellistA, 'Reference':LabellistB, 'Test_text':TrainLabel_test_text, 'Reference_text':TrainLabel_Reference_text, 'P/N':PN})

## 可以改隨機抽取的label 0個數跟模式

In [ ]:
#0的df
TrainLabel_text_0 = TrainLabel_text_all[TrainLabel_text_all['P/N'] == 0]
TrainLabel_text_0_sample = TrainLabel_text_0.sample(n=60000,random_state=123,axis=0)

In [ ]:
TrainLabel_text_sample = pd.concat([TrainLabel_text_1, TrainLabel_text_0_sample], ignore_index=True)

In [ ]:
TrainLabel_text_sample

,Test,Reference,Test_text,Reference_text,P/N
0,3,415,"['性費洛蒙', '費洛蒙', '蘇力菌', '微生物製劑', '蔥', '蔬菜', '農作...","['性費洛蒙', '費洛蒙', '蘇力菌', '窄域油', '蔥', '甘藷', '玉米',...",1
1,3,649,"['性費洛蒙', '費洛蒙', '蘇力菌', '微生物製劑', '蔥', '蔬菜', '農作...","['性費洛蒙', '費洛蒙', '蘇力菌', '蔥', '甘藷', '玉米', '大蒜', ...",1
2,9,5,"['苦楝油', '菸草水', '水稻', '菸草', '褐飛蝨', '白背飛蝨', '斑飛蝨...","['苦楝油', '菸草水', '水稻', '菸草', '農作物', '褐飛蝨', '白背飛蝨...",1
3,25,32,"['腐絕快得寧', '待克利', '百克敏', '快得寧', '腐絕', '草莓', '炭疽...","['腐絕快得寧', '待克利', '百克敏', '快得寧', '腐絕', '草莓', '炭疽...",1
4,25,41,"['腐絕快得寧', '待克利', '百克敏', '快得寧', '腐絕', '草莓', '炭疽...","['腐絕快得寧', '待克利', '百克敏', '快得寧', '腐絕', '草莓', '炭疽...",1
...,...,...,...,...,...
61376,644,1342,"['鹼性氯氧化銅', '快得寧', '多保鏈黴素', '鏈黴素', '鋅波爾多', '銅快得...","['蕉', '豆科', '雜草', '茄科', '蚜蟲', '嵌紋病', '棉蚜', '蕉蚜']",0
61377,185,521,['斜紋夜蛾'],"['賽洛寧', '可尼丁', '賽速安', '得芬諾', '水稻', '瘤野螟', '稻縱捲...",0
61378,1010,612,"['水稻', '梅', '農作物', '稻熱病', '吊狗', '病蟲害']","['三賽唑', '賽唑可濕性粉劑', '喜樂克拉', '護粒松', '嘉賜三賽唑', '賜三...",0
61379,1315,421,"['第滅寧', '陶斯松', '褔瑞松', '蕉', '雜草', '薊馬', '花薊馬']","['賓克隆', '褔多寧', '菲克利', '維利熱必斯', '貝芬同', '熱必斯', '...",0


## 可以改成你想要的label檔名

In [ ]:
TrainLabel_text_sample.to_csv('TrainLabel_text_sample.csv', header=True, index=False)

# 製作private train data

In [ ]:
#final_data = pd.read_csv('final_data_monpa.csv', header=None, index_col=0, squeeze=True).to_dict()

In [ ]:
list_key = list(final_data.keys())
listA = [val for val in list_key for i in range(len(final_data))]
listB = list_key*(len(final_data))

In [ ]:
final_data_test_text = []
final_data_Reference_text = []
for item in listA:
    final_data_test_text.append(str(final_data[item]))
for item in listB:
    final_data_Reference_text.append(str(final_data[item]))

In [ ]:
final_data_all = pd.DataFrame({'Test':listA, 'Reference':listB, 'Test_text':final_data_test_text, 'Reference_text':final_data_Reference_text})

## 可以改隨機抽取的private data個數跟模式

In [ ]:
final_data_all.to_csv('Test_data_sample.csv', header=True, index=False)

In [ ]:
final_data_all

,Test,Reference,Test_text,Reference_text
0,100,100,"['三賽唑', '賽唑可濕性粉劑', '亞賜圃', '嘉賜黴素', '甲基多保淨', '嘉賜...","['三賽唑', '賽唑可濕性粉劑', '亞賜圃', '嘉賜黴素', '甲基多保淨', '嘉賜..."
1,100,1002,"['三賽唑', '賽唑可濕性粉劑', '亞賜圃', '嘉賜黴素', '甲基多保淨', '嘉賜...","['含毒甲基丁香油', '甲基丁香油', '丁香油', '水稻', '蔬菜', '果樹', ..."
2,100,1003,"['三賽唑', '賽唑可濕性粉劑', '亞賜圃', '嘉賜黴素', '甲基多保淨', '嘉賜...","['耐克螺', '聚乙醛', '水稻', '疫病', '福壽螺', '病蟲害']"
3,100,101,"['三賽唑', '賽唑可濕性粉劑', '亞賜圃', '嘉賜黴素', '甲基多保淨', '嘉賜...","['培丹', '撲滅松', '滅松乳劑', '水稻', '桃', '螟蟲', '二化螟']"
4,100,1013,"['三賽唑', '賽唑可濕性粉劑', '亞賜圃', '嘉賜黴素', '甲基多保淨', '嘉賜...","['性費洛蒙', '費洛蒙', '蔬菜', '豆類', '雜糧', '綠肥作物', '農田'..."
...,...,...,...,...
176395,995,982,"['加保扶', '培丹', '水稻', '桃', '雜草', '水稻水象鼻蟲', '象鼻蟲'...","['水稻', '蔥', '果樹', '桃', '楊桃', '番石榴', '蔬果類', '農作..."
176396,995,983,"['加保扶', '培丹', '水稻', '桃', '雜草', '水稻水象鼻蟲', '象鼻蟲'...","['水稻', '捲葉蟲', '縱捲葉蟲', '白葉枯病', '紋枯病']"
176397,995,989,"['加保扶', '培丹', '水稻', '桃', '雜草', '水稻水象鼻蟲', '象鼻蟲'...","['滅賜松', '撲芬松', '竹筍', '竹盲椿象', '病蟲害']"
176398,995,99,"['加保扶', '培丹', '水稻', '桃', '雜草', '水稻水象鼻蟲', '象鼻蟲'...","['水稻', '桃', '稻熱病', '葉稻熱病']"


# 處理訓練集及測試集的stop word 及標點符號 #

In [ ]:
%%time
def clean_seg_list(seg_list):
    punctuations = ['+', '?', '!', '！', '，', '.', '、', ':', '：', ';', '；', ' ', '\/', '_', ',', '$', '^', 
                    '*', '~', '\""', '\\', '/', '／', '「', '」', '...', '......', '#', '(',')', '「','(',' ',')','」',
                    '『', '』', '“', '”', '「', '」', '《', '》', '-', '－', '=', '⋯', '～', '・', '·', '<', '>', '．', '%', 
                    '％']
    stop_words = [line.strip() for line in open ('stopword_chinese.txt', encoding = 'utf-8').readlines()]
    return [seg for seg in seg_list if seg[0] not in punctuations + stop_words]

def get_cleaned_seg(df):
    df['All'] = df['Test_text'] + ' ' + df['Reference_text']
    
    cleaned = []
    for line in df['All']:
        items = line.split()
        item = ' '.join(clean_seg_list(items))
        cleaned.append(item)
    df['All_text'] = cleaned
    df = df.drop('All', axis=1)

    return df
df_train = get_cleaned_seg(TrainLabel_text_sample)
df_test = get_cleaned_seg(final_data_all)

Wall time: 46.3 s


# 訓練模型 #

In [ ]:
labels = df_train['P/N'].tolist()
features = df_train['All_text'].tolist()
features_pre = df_test['All_text'].tolist()

In [ ]:
tfidf_BOW = TfidfVectorizer()
tfidf_BOW.fit(features)
kfold = KFold(n_splits = 10,shuffle = True)

In [ ]:
%%time
#RF

model = RandomForestClassifier()
expected = []
predicted_class = []
for train_index, test_index in kfold.split(features):
    x_train, x_test = np.array(features)[train_index], np.array(features)[test_index]
    y_train, y_test = np.array(labels)[train_index], np.array(labels)[test_index]

    vectors_train = tfidf_BOW.transform(x_train)
    vectors_test = tfidf_BOW.transform(x_test)

    model.fit(vectors_train, y_train)

    expected.extend(y_test)
    predicted_class.extend(model.predict(vectors_test))

print ("Classification report for classifier: \n %s" % (metrics.classification_report(expected, predicted_class)))
print ("Confusion matrix: \n%s" % metrics.confusion_matrix(expected, predicted_class))

Classification report for classifier: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     60000
           1       0.78      0.74      0.76      1381

    accuracy                           0.99     61381
   macro avg       0.89      0.87      0.88     61381
weighted avg       0.99      0.99      0.99     61381

Confusion matrix: 
[[59710   290]
 [  360  1021]]
Wall time: 6min 51s


# 預測模型 #

In [ ]:
%%time
x_pre = np.array(features_pre)
vectors_pre = tfidf_BOW.transform(x_pre)

class_pre = []
class_pre.extend(model.predict(vectors_pre))

Wall time: 9.46 s


# 存檔 #

In [ ]:
df_ans = pd.DataFrame({'ans':class_pre})
df_concat = pd.concat([df_test['Test'],df_test['Reference'],df_ans],axis = 1)

df_final = df_concat[df_concat['ans']==1]
df_submission = df_final.drop('ans',axis = 1)
df_submission.head()

,Test,Reference
1301,101,1109
1305,101,1117
3391,1021,1090
3811,1026,1090
4734,1031,1335


In [ ]:
num = []
for i in range(len(df_submission)):
    if df_submission.iloc[i,0] == df_submission.iloc[i,1]:
        num.append(i)
df_submission_drop = df_submission.drop(df_submission.index[num])

In [ ]:
df_submission_drop.to_csv('submission_example.csv',header =True,index=False)